In [105]:
import pandas as pd
import numpy as np
import statistics
from scipy.stats import f_oneway, rankdata
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import mutual_info_classif

In [2]:
# read in file
df = pd.read_csv("Brain_GSE50161.csv")

In [3]:
# first column is the samples
# second column is the types for brain cancers.
print(df['type'].value_counts())
# replace type to numbers
df['type'] = df['type'].replace(['ependymoma', 'glioblastoma','medulloblastoma', 'pilocytic_astrocytoma','normal'],[0,1,2,3,4])
df

ependymoma               46
glioblastoma             34
medulloblastoma          22
pilocytic_astrocytoma    15
normal                   13
Name: type, dtype: int64


,samples,type,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
0,834,0,12.498150,7.604868,6.880934,9.027128,4.176175,7.224920,6.085942,6.835999,...,9.979005,9.926470,12.719785,12.777792,5.403657,4.870548,4.047380,3.721936,4.516434,4.749940
1,835,0,13.067436,7.998090,7.209076,9.723322,4.826126,7.539381,6.250962,8.012549,...,11.924749,11.215930,13.605662,13.401342,5.224555,4.895315,3.786437,3.564481,4.430891,4.491416
2,836,0,13.068179,8.573674,8.647684,9.613002,4.396581,7.813101,6.007746,7.178156,...,12.154405,11.532460,13.764593,13.477800,5.303565,5.052184,4.005343,3.595382,4.563494,4.668827
3,837,0,12.456040,9.098977,6.628784,8.517677,4.154847,8.361843,6.596064,6.347285,...,11.969072,11.288801,13.600828,13.379029,4.953429,4.708371,3.892318,3.759429,4.748381,4.521275
4,838,0,12.699958,8.800721,11.556188,9.166309,4.165891,7.923826,6.212754,6.866387,...,11.411701,11.169317,13.751442,13.803646,4.892677,4.773806,3.796856,3.577544,4.504385,4.541450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,959,3,12.658228,8.843270,7.672655,9.125912,5.495477,8.603892,7.747514,5.828978,...,13.170441,12.676080,14.124837,13.996436,4.913579,4.399176,3.878855,3.680103,4.726784,4.564637
126,960,3,12.812823,8.510550,8.729699,9.104402,3.967228,7.719089,7.092496,6.504812,...,13.040267,12.403316,13.978009,13.812916,5.189600,4.912618,3.764800,3.664920,4.628355,4.761351
127,961,3,12.706991,8.795721,7.772359,8.327273,6.329383,8.550471,6.613332,6.308945,...,12.825383,12.439265,14.328373,14.008693,4.931460,4.712895,3.913637,3.700964,4.764693,4.834952
128,962,3,12.684593,8.293938,7.228186,8.494428,6.049414,8.214729,7.287758,5.732710,...,13.116581,12.657967,14.390346,14.194904,4.871092,4.739400,3.782980,3.920363,4.665584,4.613326


In [40]:
# Use ANOVA to filter genes based on p-value

stat_ANOVA = []

# loop through each genes
for g in range(2, df.shape[1]):
    
    # subset type 0 (ependymoma) for gene g
    type0 = np.array(df[df.type == 0].iloc[:,g])
    
    # subset type 1 (glioblastoma) for gene g
    type1 = np.array(df[df.type == 1].iloc[:,g])
    
    # subset type 2 (medulloblastoma) for gene g
    type2 = np.array(df[df.type == 2].iloc[:,g])
    
    # subset type 3 (pilocytic_astrocytoma) for gene g
    type3 = np.array(df[df.type == 3].iloc[:,g])
    
    # subset type 4 (normal) for gene g
    type4 = np.array(df[df.type == 4].iloc[:,g])
    
    # run 1-way between subjects ANOVA
    result = f_oneway(type0, type1, type2, type3, type4)
    
    # append p-value
    stat_ANOVA.append(result[1])
        
    if g%1000 == 0:
        print(g)
    
    

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000


In [58]:
np.quantile(stat_ANOVA, .25)

2.8735437279936423e-07

In [61]:
# return genes with p-value less than 0.001
stat_TF = [True, True]

for i in stat_ANOVA:
    if i <= 0.0000003:
        stat_TF.append(True)
    else:
        stat_TF.append(False)

In [75]:
df = df.loc[:,stat_TF]

In [76]:
df

,samples,type,1007_s_at,1053_at,1255_g_at,1294_at,1431_at,1487_at,1552256_a_at,1552269_at,...,65588_at,78495_at,823_at,87100_at,90265_at,91816_f_at,91920_at,AFFX-HSAC07/X00351_M_at,AFFX-HUMISGF3A/M97935_3_at,AFFX-HUMISGF3A/M97935_MB_at
0,834,0,12.498150,7.604868,4.176175,7.224920,5.513410,7.905140,8.386617,8.222273,...,8.227121,7.500094,7.232380,7.193072,8.178087,7.762491,10.956206,13.104597,9.104209,7.452986
1,835,0,13.067436,7.998090,4.826126,7.539381,6.173106,8.269136,9.208559,6.914568,...,9.021009,7.978950,9.822285,8.391833,10.667338,11.170625,10.255266,13.814658,10.411498,9.122687
2,836,0,13.068179,8.573674,4.396581,7.813101,6.323471,8.327897,9.344553,6.706525,...,8.395228,8.338153,12.257851,8.582821,9.574368,8.443621,10.364209,14.248673,11.321259,9.822739
3,837,0,12.456040,9.098977,4.154847,8.361843,6.008684,7.730238,8.996408,8.985858,...,9.058105,8.529769,8.526016,6.461401,8.743637,9.597551,8.451971,13.660059,9.924395,8.028984
4,838,0,12.699958,8.800721,4.165891,7.923826,5.279579,7.581397,10.040652,8.221752,...,8.123893,7.963959,9.156358,7.462321,8.123024,8.106023,10.490535,14.042310,10.786574,9.086955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,959,3,12.658228,8.843270,5.495477,8.603892,4.967369,8.666578,10.233177,4.996330,...,8.665639,8.324939,10.151682,8.639360,8.559671,9.353849,8.719159,13.713351,10.719271,8.359550
126,960,3,12.812823,8.510550,3.967228,7.719089,5.082896,8.737005,9.902292,5.379189,...,9.805503,8.478662,10.226974,7.584308,8.753927,9.382903,9.022079,13.696252,10.303234,8.257289
127,961,3,12.706991,8.795721,6.329383,8.550471,4.919414,8.724114,9.638112,4.627095,...,8.518508,8.495768,9.964082,8.383192,8.466273,8.744321,9.545324,13.097621,11.074307,8.822692
128,962,3,12.684593,8.293938,6.049414,8.214729,5.140635,8.333919,10.145374,4.917388,...,9.374640,8.235325,9.843288,9.016851,8.469799,8.441841,10.455301,13.494138,10.848698,8.689571


In [77]:
# Use variance to filter

stat_var = []

# loop through each genes
#df.shape[1]
for g in range(2, df.shape[1]):
    stat_var.append(statistics.variance(df.iloc[:,g]))
    if g%1000==0:
        print(g)
    

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


In [84]:
np.median(stat_var)

0.6434791850009995

In [86]:
var_TF = [True, True]
for i in stat_var:
    if i >= 0.6:
        var_TF.append(True)
    else:
        var_TF.append(False)

In [91]:
df = df.loc[:,var_TF]

IndexError: Boolean index has wrong length: 13706 instead of 7256

In [92]:
df

,samples,type,1007_s_at,1255_g_at,1552269_at,1552283_s_at,1552296_at,1552301_a_at,1552312_a_at,1552320_a_at,...,64408_s_at,64900_at,65472_at,65588_at,823_at,87100_at,90265_at,91816_f_at,91920_at,AFFX-HUMISGF3A/M97935_MB_at
0,834,0,12.498150,4.176175,8.222273,8.271134,7.788566,7.160071,7.363721,6.313468,...,8.278230,10.529892,6.011124,8.227121,7.232380,7.193072,8.178087,7.762491,10.956206,7.452986
1,835,0,13.067436,4.826126,6.914568,6.503762,6.207202,6.529700,9.629058,5.273361,...,6.605263,10.663373,6.992694,9.021009,9.822285,8.391833,10.667338,11.170625,10.255266,9.122687
2,836,0,13.068179,4.396581,6.706525,7.446118,7.390576,8.191684,7.989959,4.942637,...,5.947323,10.608897,7.453627,8.395228,12.257851,8.582821,9.574368,8.443621,10.364209,9.822739
3,837,0,12.456040,4.154847,8.985858,8.321482,8.866840,5.758467,7.954845,6.715825,...,6.776072,11.796334,9.159088,9.058105,8.526016,6.461401,8.743637,9.597551,8.451971,8.028984
4,838,0,12.699958,4.165891,8.221752,7.407162,7.385834,7.042512,8.816403,4.623536,...,7.538854,9.672255,6.077686,8.123893,9.156358,7.462321,8.123024,8.106023,10.490535,9.086955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,959,3,12.658228,5.495477,4.996330,6.453617,5.224920,6.133921,6.570050,3.573504,...,6.043274,9.126857,6.987462,8.665639,10.151682,8.639360,8.559671,9.353849,8.719159,8.359550
126,960,3,12.812823,3.967228,5.379189,7.177535,5.739474,6.679918,7.773944,4.057522,...,6.060871,9.211240,6.916661,9.805503,10.226974,7.584308,8.753927,9.382903,9.022079,8.257289
127,961,3,12.706991,6.329383,4.627095,7.224081,5.728723,6.233379,7.181821,3.616429,...,6.108193,9.568512,7.491359,8.518508,9.964082,8.383192,8.466273,8.744321,9.545324,8.822692
128,962,3,12.684593,6.049414,4.917388,7.966999,5.892257,5.912674,6.194283,3.449454,...,5.646451,9.910713,7.431232,9.374640,9.843288,9.016851,8.469799,8.441841,10.455301,8.689571


In [99]:
# Use Information Gain to filter

#x is columns for genes
#y is the cancer type
X = np.array(df.iloc[:,2:])
y = np.array(df.iloc[:,1])

IG = mutual_info_classif(X,y,n_neighbors = 5)
print("IG done")

IG done


In [146]:
# Select top 3000 feature based on importance from Information Gain
rank = rankdata(IG)
rank_TF = [True, True]
for i in rank:
    if i <= 3000:
        rank_TF.append(True)
    else:
        rank_TF.append(False)

df = df.loc[:,rank_TF]
df

,samples,type,1255_g_at,1552283_s_at,1552301_a_at,1552312_a_at,1552417_a_at,1552439_s_at,1552455_at,1552508_at,...,50965_at,52837_at,53071_s_at,53720_at,53991_at,60471_at,65472_at,65588_at,87100_at,AFFX-HUMISGF3A/M97935_MB_at
0,834,0,4.176175,8.271134,7.160071,7.363721,6.285466,8.195263,5.327724,5.989892,...,7.243988,7.153611,8.183152,7.755687,8.856580,7.203817,6.011124,8.227121,7.193072,7.452986
1,835,0,4.826126,6.503762,6.529700,9.629058,8.558855,9.598322,5.573461,8.319513,...,11.028732,7.380205,9.191431,9.143043,6.764527,8.420093,6.992694,9.021009,8.391833,9.122687
2,836,0,4.396581,7.446118,8.191684,7.989959,7.505988,7.291341,6.769834,7.261455,...,7.407671,6.468495,9.263549,9.033832,8.960278,8.414553,7.453627,8.395228,8.582821,9.822739
3,837,0,4.154847,8.321482,5.758467,7.954845,7.635059,8.822038,4.465923,5.883377,...,6.961235,6.381614,10.339334,9.352513,10.307893,7.587536,9.159088,9.058105,6.461401,8.028984
4,838,0,4.165891,7.407162,7.042512,8.816403,7.513383,6.542362,7.241639,6.665956,...,7.910449,5.703983,8.723818,8.599707,9.719175,8.082934,6.077686,8.123893,7.462321,9.086955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,959,3,5.495477,6.453617,6.133921,6.570050,6.214122,9.824718,5.501943,7.563709,...,7.727297,5.852597,10.482143,9.204577,9.400551,9.012567,6.987462,8.665639,8.639360,8.359550
126,960,3,3.967228,7.177535,6.679918,7.773944,6.895118,9.077885,6.835527,10.704805,...,7.695190,5.682251,9.311126,8.791367,9.609665,8.549925,6.916661,9.805503,7.584308,8.257289
127,961,3,6.329383,7.224081,6.233379,7.181821,5.778619,12.290301,5.307350,6.450710,...,9.019292,5.552393,10.290373,9.793867,9.719978,9.468578,7.491359,8.518508,8.383192,8.822692
128,962,3,6.049414,7.966999,5.912674,6.194283,5.761735,12.246204,7.851377,7.456670,...,9.811593,5.495569,9.652443,9.497484,9.914592,9.582776,7.431232,9.374640,9.016851,8.689571


In [ ]:
#x is columns for genes (features)
#y is the cancer type
# x = df.iloc[:,2:]
# y = df.iloc[:,1]

In [ ]:
#wrappered method
# model = GaussianMixture(n_components=5)
# print("model done")
# gmm = model.fit(x)
#wrappered method
# model = GaussianMixture(n_components=2)
# print("model done")
# gmm = model.fit(x)
# print("fit done")
###Due to the large time complexity of Gaussian Mixture model, we can do the linear regression at first.
# model = LinearRegression()
# rfe = RFE(model, n_features_to_select=500)
# #Use RFE to select the top 5 features 
# fit = rfe.fit(x, y)
# print('fit done')